In [2]:
import numpy as np 
import math, matplotlib, time
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.offsetbox import AnchoredText
import pandas as pd
import os, glob, datetime
from scipy.interpolate import griddata
matplotlib.rc('font',size=16)
# %matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('./')
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
# from plotly.subplots import make_subplots

import pdfchem
import mercury as mr

def screen_time():
    return datetime.datetime.now().strftime("\033[0;32m%H:%M:%S %Y-%m-%d > \33[0m")

In [31]:
app = mr.App(title="PDFchem 3.0 beta 3 - Plotly", description="interactive PDF analysis")
max_workers = 1

mercury.App

In [32]:
# Hello in PDFchem! 👋

In [33]:
plot_input_PDF=False

PDF_input_flag = mr.Checkbox(value=False, label="Switch on to use your PDF files", url_key="flag")
if not PDF_input_flag.value:
    metallicity = mr.Select(value="1", choices=["0.1", "0.5", "1", "2"], label='Metallicity:')
    _mean = mr.Numeric(value=4, min=0.6, max=20, step=0.2, label=r'Mean: Av_bar')
    _width = mr.Numeric(value=0.32, min=0.12, max=1.02, step=0.1, label='Width: Sigma')
    output_file = 'output.dat'
    run_model_button = mr.Button(label="Run model PDF", style="primary")

if PDF_input_flag.value:
    plot_input_PDF=True
    multi_PDF_flag = mr.Checkbox(value=False, label="Switch on to run multiple Av-PDF files", url_key="multi PDF flag")
    if multi_PDF_flag.value:
        #PDF_path = my_PDF.filepath
        PDF_path = mr.Text(value="./avpdf_input/", label="Enter a folder containing your Av-PDF files", rows=1)
        avpdf_files = sorted(glob.glob(os.path.join(PDF_path.value,'*')))
        mr.Md(f"## WARNING: will run PDFchem with all files in `{PDF_path.value}`!")
        mr.Md(f"## This could be very slow if you have many files!")
        metallicity = mr.Select(value="1", choices=["0.1", "0.5", "1", "2"], label='Metallicity:')
        run_multi_button = mr.Button(label="Run multiple PDF", style="primary")
        my_PDf = avpdf_files[0]
    if not multi_PDF_flag.value :
        my_PDF = mr.File(label="Please upload an Av_PDF file", max_file_size="100MB")
        metallicity = mr.Select(value="1", choices=["0.1", "0.5", "1", "2"], label='Metallicity:')
        run_single_button = mr.Button(label="Run a single PDF", style="primary")
        

mercury.Checkbox

mercury.Select

mercury.Numeric

mercury.Numeric

mercury.Button

In [34]:
av_diff, pdf_diff, avtot_diff = pdfchem.makepdf(1.4, 0.42)
av_sf, pdf_sf, avtot_sf = pdfchem.makepdf(7, 0.32)

fig = go.Figure()
fig.add_trace(go.Scatter(x=av_diff, y=pdf_diff,
                    mode='lines',line=dict(color='LightSeaGreen', width=2, dash = 'dash'), hovertemplate = ' ',
                    name='quiescent'))
fig.add_trace(go.Scatter(x=av_sf, y=pdf_sf,
                    mode='lines', line=dict(color='RebeccaPurple ', width=2, dash = 'dashdot'),hovertemplate = ' ',
                    name='star-forming'))
if not PDF_input_flag.value:
    av_bar, s, metallicity = _mean.value, _width.value, float(metallicity.value)
    av_m, pdf_m, avtot = pdfchem.makepdf(av_bar, s)
    fig.add_trace(go.Scatter(x=av_m, y=pdf_m, mode='lines', hovertemplate = ' ',
    line = dict(color='Crimson', width=4), name='model PDF'))
elif my_PDF.filename:
    try:
        d = np.genfromtxt(my_PDF.filepath)  # user specified PDF
        av = d[:,0]; pdf = d[:,1]
        fig.add_trace(go.Scatter(x=av_m, y=pdf_m, mode='lines', line = dict(color='Crimson', width=4), name='user PDF'), hovertemplate = ' ',)
    except:
        if my_PDF.filename[-4:] == 'fits':
            av, pdf = pdfchem.NH_fits_to_pdf(my_PDF.filepath)
            fig.add_trace(go.Scatter(x=av_m, y=pdf_m, mode='lines', line = dict(color='Crimson', width=4), name='user PDF'))
fig.update_xaxes(title_text='<i>A</i><sub>V</sub> [mag]', type='log', range = [-1, 2], nticks=4, title_font_family='Rockwell')
fig.update_yaxes(title_text='<i>p</i>', type='log', range = [-3, 0.5], nticks=4, title_font_family='Rockwell')

fig.update_layout(width = 520, height = 300,
                   margin=dict(l=50, r=50, b=50, t=25),
                   font=dict(family="Rockwell",size=15,color="black"),
                   hoverlabel_font=dict(family="Rockwell",size=18,color="white"),
                   legend=dict(orientation="h", yanchor="bottom", y=1.02,),
                   )
fig.show()

In [7]:
# fig.write_image('figures/PDFs.pdf')

In [8]:
plot_species = mr.Select(value="Tco10", choices=[
"Tgas", "CII", "CI",  "CO", "OI",  "HI",  "H2", 'HI_H2_ratio', "OHp",  "H2Op",  "OH", "CH",  "HCOp", "Tcii",  "Tci10",  "Tci21", "Tco10", "Tco21",  "Tco32", "Tco43",  "Tco54",  "Tco65", "Tco76",  "Tco87",  "Tco98", "Tco109" 
], label="Select a specie to plot the result.")

mercury.Select

In [9]:
ref_value_flag = mr.Checkbox(value=False, label="Switch on to enter a reference value", url_key="flag")
if ref_value_flag.value:
    ref_value = mr.Numeric(value=1, min=0, max=1e12, label="Enter a reference value to show on the result figure")

mercury.Checkbox

In [10]:
make_collective_figs_button = mr.Button(label="Show collective figures", style="primary")
make_carbon_figs_button = mr.Button(label="Show carbon cycle figures", style="primary")

mercury.Button

mercury.Button

In [11]:
#colour map setup
cmap = plt.cm.get_cmap("viridis")

my_cmap = cmap(np.arange(cmap.N))
alphas = np.linspace(0.6,0.6,cmap.N)
BG = np.asarray([1.,1.,1.])
for i in range(cmap.N):
    my_cmap[i,:-1] = my_cmap[i,:-1] * alphas[i] + BG*(1.-alphas[i])
my_cmap = ListedColormap(my_cmap)
cmap=my_cmap

In [12]:
if not PDF_input_flag.value:
    writeout = np.column_stack((_mean.value, _width.value, metallicity))
    np.savetxt('pdfchem.params',writeout,delimiter=' ',fmt='%1.4e')

def process_file(avpdf_file):
    file_index =  avpdf_file.split('/')[-1].split('.')[0][-6:]
    pdfchem_output_file = os.path.join(f'./pdfchem_output', f'output{file_index}.dat')
    pdfchem.main(avpdf_file, pdfchem_output_file)
    return

startTime = time.time()

if run_model_button.clicked:
    pdfchem.main()
    executionTime = round(time.time() - startTime, 1)
    print(f'Execution time: {executionTime} seconds.')
else:
    try: 
        if run_single_button.clicked:    pdfchem.main(my_PDF.filepath)
        executionTime = round(time.time() - startTime, 1)
        print(f'Execution time: {executionTime} seconds.')
    except: pass
    try: 
        if run_multi_button.clicked:  
            print(f'{screen_time()} \033[0;31mPDFchem \33[0m started.')
            if not os.path.exists('./pdfchem_output'): os.mkdir('./pdfchem_output')
            for avpdf_file in avpdf_files:
                process_file(avpdf_file)
            output_file = sorted(glob.glob(os.path.join('pdfchem_output','*.dat')))[0]
            print(f'{screen_time()} \033[0;31m{avpdf_file}\33[0m completed')
            executionTime = round(time.time() - startTime, 1)
            print(f'Execution time: {executionTime} seconds.')
    except: pass

# 45    seconds with i7-1165G7 @ 2.80GHz with Multiprocessing ON
# 15.7  seconds with AMD Ryzen 9 5900HX
# 9.5   seconds with Xeon(R) Silver 4214R
# 8.1   seconds with M1 pro
# 2.5   seconds with Xeon(R) Platinum 8358


Execution time in seconds: 0.0


In [13]:
# read the output from the algorithm
title = r'PDFchem result'

In [14]:
# mr.Md(f"""
# ## Sequence of species
# |||||||||||
# |--------|---------|----------|---------|---------|-------|--------|---------|---------|-------|
# | 1: H3+ | 2: He+  | 3: Mg    | 4: H2+  | 5: CH5+ | 6: O2 |7: CH4+ |8: O+    |9: OH+   |10: Mg+|
# |11: C+  |12: CH4  |13: H2O+  |14: H3O+ |15: CO+  |21: CH |22: CH3 |23: HCO+ |24: CH2+ |25: C  | 
# |26: He  |27: CH+  |28: CO    |29: OH   |30: O    |31: H2 |32: H   |33: e-   |         |       |
# ## Results will be placed in `./pdfchem_output`
# """)

In [15]:
#CHEMISTRY RESULTS
try:
    d = np.genfromtxt(output_file)

    #UV radiation (Draine); cosmic-ray ionization rate (s-1); metallicity (Zsolar)
    UV = d[:,0]; CR = d[:,1]; Z = d[:,2]
    #Gas temperature
    Tgas = d[:,3]
    #Abundances of CII; CI; CO; OI; HI; H2; OH+; H2O+; OH; CH; HCO+
    CII = d[:,14]; CI = d[:,28]; CO = d[:,31]
    OI = d[:,33]; HI = d[:,35]; H2 = d[:,34]
    OHp = d[:,12]; H2Op = d[:,16]; OH = d[:,32]
    CH = d[:,24]; HCOp = d[:,26]
    HI_H2_ratio = HI/H2/2
    #the above are some of the available species
    #follow the sequence of species by adding 3 to plot any other available

    #Brightness temperatures of [CII]158; [CI](1-0); [CI](2-1)
    Tcii = d[:,37]; Tci10 = d[:,38]; Tci21 = d[:,39]
    #CO(1-0); CO(2-1); ... CO(10-9)
    Tco10 = d[:,40]; Tco21 = d[:,41]; Tco32 = d[:,42] 
    Tco43 = d[:,43]; Tco54 = d[:,44]; Tco65 = d[:,45]
    Tco76 = d[:,46]; Tco87 = d[:,47]; Tco98 = d[:,48]
    Tco109 = d[:,49]
except:
    pass   

#AV-PDF 
try:
    d = np.genfromtxt(avpdf_file)  # user specified PDF
    av = d[:,0]; pdf = d[:,1]
    av_bar = 'user AV-PDF'
    s = '*'
    #av = 10**d[:,0]
except:
    d = np.genfromtxt('avpdf.dat')       # program generated PDF

myVars = globals()
species = myVars[plot_species.value]
#for plotting the grid
xi = np.linspace(min(np.log10(CR)), max(np.log10(CR)), 50)
yi = np.linspace(min(np.log10(UV)), max(np.log10(UV)), 50)
points = np.array([np.log10(CR),np.log10(UV)]).T

In [16]:
qty = species # e.g. 'CO' for CO abundance, 'Tco21/Tco10' for CO(2-1)/CO(1-0)
title = f'lg {plot_species.value}'
if title == 'lg HI_H2_ratio': title = f'lg HI_H2_ratio'

zi = griddata((np.log10(CR),np.log10(UV)),np.log10(qty),(xi[None,:], yi[:,None]),method='linear')

fig1 = go.Figure()
fig1.add_trace(go.Contour(z=zi,x=xi ,y=yi,
            contours=dict(coloring ='heatmap', showlabels = True, labelfont = dict(size = 22,color = 'Black')),
            colorbar=dict( titleside='right', titlefont=dict(size=22, family='Rockwell'))))
fig1.update_traces(name='',
                   customdata=10**zi, line_dash='dashdot', line_width=0.5,
                   hovertemplate = "<b>%{customdata:.3g}</b>"+
                   "<br>lg <b>UV</b>=%{y:.1f}"+
                   "<br>lg <b>CR</b>=%{x:.1f}",
                   colorscale='Aggrnyl', colorbar_tickprefix=''
                   )
try:
    ref_val = np.log10(ref_value.value)
    fig1.add_trace(go.Contour(z=zi, x=xi ,y=yi, contours_coloring='lines',line_width=4, hoverinfo='none', showscale=False, contours=dict(coloring ='lines',start=ref_val, end=ref_val,size=5)))
except:
    pass

fig1.update_xaxes(title_text='log<sub>10</sub><i>&#950;</i><sub>CR</sub> [s<sup>-1</sup>]', nticks=5, ticks='outside', automargin=True)
fig1.update_yaxes(title_text='log<sub>10</sub> <i>&#967;/&#967;<sub>0</sub></i>', ticks='outside', scaleanchor = "x", scaleratio = 4/6)

fig1.update_layout(width = 570, height = 520,
                   hovermode="x", margin=dict(l=50, r=50, b=50, t=50),
                   font=dict(family="Rockwell",size=22,color="black"), title={'text': title, 'x':0.95, 'font_size': 22}, 
                   hoverlabel_font=dict(family="monospace",size=18,color="white")
                   )
fig1.show()

In [17]:
# fig1.write_image(f'figures/{title}.pdf')

In [18]:
def grid_data(values):
    try:
        zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(values),xi,yi,interp='linear') 
    except:
        zi = griddata(points,np.log10(values),(xi,yi),method='linear') 
    return(zi)

In [19]:
if make_collective_figs_button.clicked:
#Collective plot similar to paper

    fig, axs = plt.subplots(4, 5, figsize=(20,19))

    xt = -16.85; yt = -0.65; N=6
    try:
        zi_tr = ml.griddata(np.log10(CR),np.log10(UV),np.log10(HI/2./H2),xi,yi,interp='linear') 
        zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(qty),xi,yi,interp='linear')
    except:        # in newer versions of matplotlib there isn't mlab.griddata(), so use scipy instead:
        xi,yi = np.mgrid[min(np.log10(CR)):max(np.log10(CR)):50j, min(np.log10(UV)):max(np.log10(UV)):50j]
        zi_tr = griddata(points,np.log10(HI/2./H2),(xi,yi),method='linear') 

    #zi_tr = grid_data(HI/2./H2)
    axs[0,0].pcolormesh(xi,yi,zi_tr,cmap=cmap)
    CS = axs[0,0].contour(xi,yi,zi_tr,N,colors='k')
    axs[0,0].clabel(CS,fmt='%1.1f')
    CS = axs[0,0].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    text_box = AnchoredText("a", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[0,0].add_artist(text_box)
    axs[0,0].set_xticks([])
    axs[0,0].set_yticks([-1, 0, 1, 2, 3, 4, 5])
    axs[0,0].set_ylabel(r'$\log\,\,\chi/\chi_0$')
    axs[0,0].set_title('HI-to-H2')

    #zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(CII),xi,yi,interp='linear')
    zi = grid_data(CII)
    axs[0,1].pcolormesh(xi,yi,zi,cmap=cmap)
    CS = axs[0,1].contour(xi,yi,zi,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    CS = axs[0,1].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    text_box = AnchoredText("b", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[0,1].add_artist(text_box)
    axs[0,1].set_xticks([])
    axs[0,1].set_yticks([])
    axs[0,1].set_title('CII abundance')

    #zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(CI),xi,yi,interp='linear')
    zi = grid_data(CI)
    axs[0,2].pcolormesh(xi,yi,zi,cmap=cmap)
    CS = axs[0,2].contour(xi,yi,zi,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    CS = axs[0,2].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    text_box = AnchoredText("c", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[0,2].add_artist(text_box)
    axs[0,2].set_xticks([])
    axs[0,2].set_yticks([])
    axs[0,2].set_title('CI abundance')

    #zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(CO),xi,yi,interp='linear')
    zi = grid_data(CO)
    axs[0,3].pcolormesh(xi,yi,zi,cmap=cmap)
    CS = axs[0,3].contour(xi,yi,zi,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    CS = axs[0,3].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    text_box = AnchoredText("d", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[0,3].add_artist(text_box)
    axs[0,3].set_xticks([])
    axs[0,3].set_yticks([])
    axs[0,3].set_title('CO abundance')

    #zi = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tgas),xi,yi,interp='linear')
    zi = grid_data(Tgas)
    axs[0,4].pcolormesh(xi,yi,zi,cmap=cmap)
    CS = axs[0,4].contour(xi,yi,zi,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    CS = axs[0,4].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    text_box = AnchoredText("e", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[0,4].add_artist(text_box)
    axs[0,4].set_xticks([])
    axs[0,4].set_yticks([])
    axs[0,4].set_title(r'log T$_{\rm gas}$ [K]')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(OH),xi,yi,interp='linear')
    zi_R = grid_data(OH)
    axs[1,0].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[1,0].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[1,0].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[1,0].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("f", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[1,0].add_artist(text_box)
    axs[1,0].set_xticks([])
    axs[1,0].set_yticks([-1, 0, 1, 2, 3, 4, 5])
    axs[1,0].set_ylabel(r'$\log\,\,\chi/\chi_0$')
    axs[1,0].set_title('OH abundance')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(OHp),xi,yi,interp='linear')
    zi = grid_data(OHp)
    axs[1,1].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[1,1].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[1,1].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[1,1].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("g", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[1,1].add_artist(text_box)
    axs[1,1].set_xticks([])
    axs[1,1].set_yticks([])
    axs[1,1].set_title(r'OH$^+$ abundance')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(H2Op),xi,yi,interp='linear')
    zi_R = grid_data(H2Op)
    axs[1,2].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[1,2].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[1,2].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[1,2].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("h", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[1,2].add_artist(text_box)
    axs[1,2].set_xticks([])
    axs[1,2].set_yticks([])
    axs[1,2].set_title(r'H$_2$O$^+$ abundance')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(CH),xi,yi,interp='linear')
    zi_R = grid_data(CH)
    axs[1,3].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[1,3].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[1,3].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[1,3].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("i", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[1,3].add_artist(text_box)
    axs[1,3].set_xticks([])
    axs[1,3].set_yticks([])
    axs[1,3].set_title('CH abundance')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(HCOp),xi,yi,interp='linear')
    zi_R = grid_data(HCOp)
    axs[1,4].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[1,4].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[1,4].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[1,4].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("j", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[1,4].add_artist(text_box)
    axs[1,4].set_xticks([])
    axs[1,4].set_yticks([])
    axs[1,4].set_title(r'HCO$^+$ abundance')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tci21/Tci10),xi,yi,interp='linear')
    zi_R = grid_data(Tci21/Tci10)
    axs[2,0].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[2,0].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[2,0].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[2,0].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("k", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[2,0].add_artist(text_box)
    axs[2,0].set_xticks([])
    axs[2,0].set_yticks([-1, 0, 1, 2, 3, 4, 5])
    axs[2,0].set_ylabel(r'$\log\,\,\chi/\chi_0$')
    axs[2,0].set_title('[CI](2-1) / [CI](1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tco21/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tco21/Tco10)
    axs[2,1].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[2,1].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[2,1].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[2,1].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("l", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[2,1].add_artist(text_box)
    axs[2,1].set_xticks([])
    axs[2,1].set_yticks([])
    axs[2,1].set_title('CO(2-1) / CO(1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tco32/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tco32/Tco10)
    axs[2,2].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[2,2].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[2,2].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[2,2].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("m", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[2,2].add_artist(text_box)
    axs[2,2].set_xticks([])
    axs[2,2].set_yticks([])
    axs[2,2].set_title('CO(3-2) / CO(1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tco43/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tco43/Tco10)
    axs[2,3].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[2,3].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[2,3].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[2,3].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("n", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[2,3].add_artist(text_box)
    axs[2,3].set_xticks([])
    axs[2,3].set_yticks([])
    axs[2,3].set_title('CO(4-3) / CO(1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tco54/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tco54/Tco10)
    axs[2,4].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[2,4].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[2,4].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[2,4].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("o", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[2,4].add_artist(text_box)
    axs[2,4].set_xticks([])
    axs[2,4].set_yticks([])
    axs[2,4].set_title(r'CO(5-4) / CO(1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tci21/Tco76),xi,yi,interp='linear')
    zi_R = grid_data(Tci21/Tco76)
    axs[3,0].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[3,0].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[3,0].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[3,0].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("p", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[3,0].add_artist(text_box)
    axs[3,0].set_xticks([-17, -16, -15, -14])
    axs[3,0].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')
    axs[3,0].set_yticks([-1, 0, 1, 2, 3, 4, 5])
    axs[3,0].set_title(r'[CI](2-1) / CO(7-6)')
    axs[3,0].set_ylabel(r'$\log\,\,\chi/\chi_0$')
    axs[3,0].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tci10/Tco43),xi,yi,interp='linear')
    zi_R = grid_data(Tci10/Tco43)
    axs[3,1].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[3,1].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[3,1].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[3,1].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("q", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[3,1].add_artist(text_box)
    axs[3,1].set_xticks([-17, -16, -15, -14])
    axs[3,1].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')
    axs[3,1].set_yticks([])
    axs[3,1].set_title(r'[CI](1-0) / CO(4-3)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tci10/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tci10/Tco10)
    axs[3,2].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[3,2].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[3,2].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[3,2].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("r", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[3,2].add_artist(text_box)
    axs[3,2].set_xticks([-17, -16, -15, -14])
    axs[3,2].set_yticks([])
    axs[3,2].set_title(r'[CI](1-0) / CO(1-0)')
    axs[3,2].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tcii/Tci10),xi,yi,interp='linear')
    zi_R = grid_data(Tcii/Tci10)
    axs[3,3].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[3,3].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[3,3].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[3,3].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("s", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[3,3].add_artist(text_box)
    axs[3,3].set_xticks([-17, -16, -15, -14])
    axs[3,3].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')
    axs[3,3].set_yticks([])
    axs[3,3].set_title(r'[CII] / [CI](1-0)')

    #zi_R = ml.griddata(np.log10(CR),np.log10(UV),np.log10(Tcii/Tco10),xi,yi,interp='linear')
    zi_R = grid_data(Tcii/Tco10)
    axs[3,4].pcolormesh(xi,yi,zi_R,cmap=cmap)
    CS = axs[3,4].contour(xi,yi,zi_R,N,colors='k')
    plt.clabel(CS,fmt='%1.1f')
    axs[3,4].contour(xi,yi,zi_tr,[0],colors='r',linewidths=3.0)
    axs[3,4].contour(xi,yi,zi_R,[0],colors='w',linewidths=3.0)
    text_box = AnchoredText("t", prop=dict(fontweight="bold",fontsize=20), frameon=True, loc=4, pad=0.3)
    plt.setp(text_box.patch, boxstyle="round", facecolor='white', alpha=0.9)
    axs[3,4].add_artist(text_box)
    axs[3,4].set_xticks([-17, -16, -15, -14, -13])
    axs[3,4].set_xlabel(r'$\log\,\,\zeta_{\rm CR}$ [s$^{-1}$]')
    axs[3,4].set_yticks([])
    axs[3,4].set_title(r'[CII] / CO(1-0)')


    plt.subplots_adjust(wspace=0.05, hspace=0.2)
    #plt.suptitle(title, y=0.93, weight='bold')

    #collective_fig_path = os.path.join('Fig_Collective', 'Collective_{}.png'.format(pdf_i))
    #plt.savefig(collective_fig_path, bbox_inches='tight')

In [21]:
if  make_carbon_figs_button.clicked:
    # pio.renderers.default = "plotly_mimetype+notebook_connected"
    d = pd.read_table(output_file, header=None, sep='\s+')
    d.rename(columns= {
            0: 'UV', 1: 'CR', 2: 'Z', 3: 'Tgas', 14: 'CII', 28: 'CI', 31: 'CO', 33: 'OI', 35: 'HI', 34: 'H2', 12: 'OHp', 16: 'H2Op', 32: 'OH', 24: 'CH', 26: 'HCOp', },
            inplace=True)
    CII = d['CII']; CI = d['CI']; CO = d['CO']
    tot_carbon = CII+CI+CO
    c2 = CII/tot_carbon
    c1 = CI/tot_carbon
    co = CO/tot_carbon

    carbon_color = [''] * len(d)
    uv_cr_label  = [''] * len(d)
    for i in range(0,len(d)):
        carbon_color[i] =f"rgba({c2.iloc[i]:.3f}, {c1.iloc[i]:.3f}, {co.iloc[i]:.3f}, 0.9)"
        uv_cr_label[i]  = f"UV={math.log10(d['UV'][i]):.1f}, CR={math.log10(d['CR'][i]):.1f}"
    df = pd.DataFrame({'CI':c1, 'C+':c2, 'CO':co, 'carbon_color':carbon_color, 'uv_cr_label':uv_cr_label})

    fig = px.scatter_ternary(df, a='CI', b='CO', c='C+', hover_name='uv_cr_label',
        hover_data={'C+':':.0%', 'CI':':>.0%', 'CO':':>.0%'},
        labels={"C+": "C<sup>+</sup>", "CI": 'CI', "CO": 'CO' },
        )
    fig.update_ternaries(
        aaxis_tickformat='.0%',baxis_tickformat='.0%',caxis_tickformat='.0%',
        aaxis_title_font_color='forestgreen',
        baxis_title_font_color='blue',
        caxis_title_font_color='crimson',
        )

    fig.update_traces(
        hoverlabel=dict( bgcolor="white", font_size=16, align='left', font_family="monospace"),
        marker=dict(size=8, color = carbon_color, symbol=137),selector=dict(type='scatterternary'))
    fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    width = 620, height = 510,
    font_size=22,font_family="Rockwell"
    )

    fig.show()


In [24]:
# fig.write_image(f'figures/ternary.pdf')